In [ ]:
from pathlib import Path
import json
import subprocess
import shlex

# Step 1: Clean cache

What I did was to set a new conan cache

```shell
mkdir ~/.conan/bump_deps
conan config set storage.path="./bump_deps" 
```

Then I cmake-configured a Release and a Debug build on the M1 mac (so it'll pick up most of build dependencies too since packages aren't readily available)

# Supporting Classes

In [ ]:
class PkgInfo:
    @staticmethod
    def from_metadata(metadata_path):
        name, version, user, channel = p.relative_to(CONAN_CACHE).parent.parts

        with open(p, "r") as f:
            data = json.load(f)
            revision = data["recipe"]["revision"]

        return PkgInfo(name=name, version=version, user=user, channel=channel, revision=revision)

    @staticmethod
    def from_str(reference):
        n, revision = reference.split("#")
        if "@" in n:
            name_version, user_channel = n.split("@")
        else:
            name_version = n
            user_channel = ""

        name, version = name_version.split("/")
        if user_channel:
            user, channel = user_channel.split("/")
        else:
            user, channel = (None, None)

        return PkgInfo(name=name, version=version, user=user, channel=channel, revision=revision)

    def __init__(self, name, version, user, channel, revision):
        self.name = name
        self.version = version
        self.user = None
        if user is not None and user != "_":
            self.user = user
        self.channel = None
        if channel is not None and channel != "_":
            self.channel = channel

        self.revision = revision

        self.remote = "conancenter"
        if self.name == "ruby_installer":
            self.remote = "bincrafters"
        elif self.name == "openstudio_ruby":
            self.remote = "nrel"

    def search_packages(
        self, verbose=True, skip_shared=False, local_cache=False, arch_only=None, compiler_version_only=None
    ):
        """Filters out packages (such as Windows MSVC 15)

        Args:
        ------

        * skip_shared (bool): Don't keep the shared ones
        * local_cache (bool, default False): if True, will search your cache. Otherwise will look in self.remote

        * arch_only (None or str): if specified, will keep only this arch (eg: 'x86')
        * compiler_version_only (None or str): if specified, will keep only this compiler.version (eg: '17')

        Example with boost:
        --------------------

        pkg_info = PkgInfo(name='boost', version="1.79.0", user=None, channel=None, revision='f664bfe40e2245fa9baf1c742591d582')

        # Download everything
        pkg_info.download_all()

        !du -sh /Users/julien/.conan/bump_deps/boost/1.79.0/_/_/package/
        21G boost/1.79.0/_/_/package/

        # Filter, but keep shared=True ones
        pkg_info.cleanup_skipped_packages(skip_shared=False)

        !du -sh /Users/julien/.conan/bump_deps/boost/1.79.0/_/_/package/
        11G boost/1.79.0/_/_/package/

        # Remove the shared=True ones
        pkg_info.cleanup_skipped_packages(skip_shared=False)

        !du -sh /Users/julien/.conan/bump_deps/boost/1.79.0/_/_/package/
        6.6G boost/1.79.0/_/_/package/
        """
        json_p = Path(f"{self.name}.json")
        args = ["conan", "search", "--json", str(json_p)]
        if not local_cache:
            args += ["-r", self.remote]
        args += [self.reference()]
        if verbose:
            print(args)
        subprocess.check_call(args, stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
        with open(json_p, "r") as f:
            data = json.load(f)
        json_p.unlink(missing_ok=False)  # remove tmp json

        packages = data["results"][0]["items"][0]["packages"]
        keep_packages = []
        skipped_packages = []
        for p in packages:
            settings = p["settings"]
            os_ = settings.get("os", None)
            compiler_version = settings.get("compiler.version", None)
            compiler = settings.get("compiler", None)
            libcxx = settings.get("compiler.libcxx", None)

            if arch_only is not None:
                arch = settings.get("arch", None)
                if arch not in [arch_only, None]:
                    if verbose:
                        print(f"Skipping package with arch {arch} for os {os_}, {compiler=} for pkg {self.name}")
                    skipped_packages.append(p)
                    continue
            if compiler_version_only is not None:
                if compiler_version not in [compiler_version_only, None]:
                    if verbose:
                        print(
                            f"Skipping package with compiler.version {compiler_version} for os {os_}, {compiler=} for pkg {self.name}"
                        )
                    skipped_packages.append(p)
                    continue

            is_shared = p["options"].get("shared", None) == "True"
            if is_shared and skip_shared:
                if verbose:
                    print(f"Skipping SHARED package for os {os_}, {compiler=} for pkg {self.name}")
                skipped_packages.append(p)
                continue

            if os_ == "Windows":
                if compiler_version not in ["16", "17", None]:
                    if verbose:
                        print(f"Skipping Windows {compiler_version=} for pkg {self.name}")
                    skipped_packages.append(p)
                    continue

                runtime = settings.get("compiler.runtime", None)
                if runtime not in ["MD", "MDd", None]:
                    if verbose:
                        print(f"Skipping Windows {runtime=} for pkg {self.name}")
                    skipped_packages.append(p)
                    continue
            elif os_ == "Linux":
                if compiler not in ["gcc", "clang", None]:
                    if verbose:
                        print(f"Skipping Linux {compiler=} for pkg {self.name}")
                    skipped_packages.append(p)
                    continue

                if libcxx not in ["libstdc++11", "libc++", None]:
                    if verbose:
                        print(f"Skipping Linux {libcxx=} for pkg {self.name} with ({compiler=})")
                    skipped_packages.append(p)
                    continue

                if compiler == "gcc":
                    if compiler_version not in ["7", "8", "9", "10", "11", "12", None]:
                        if verbose:
                            print(f"Skipping Linux gcc {compiler_version=} for pkg {self.name}")
                        skipped_packages.append(p)
                        continue

            elif os_ == "Macos":
                if libcxx not in ["libc++", None]:
                    if verbose:
                        print(f"Skipping Macos {libcxx=} for pkg {self.name} with ({compiler=})")
                    skipped_packages.append(p)
                    continue
            elif os_ is None:
                pass
            else:
                print("Unknown os: {os_}")
                skipped_packages.append(p)
                continue

            keep_packages.append(p)

        return keep_packages, skipped_packages

    def download_all(self):
        subprocess.check_call(
            ["conan", "download", "-r", self.remote, self.reference()],
            # stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
        )

    def download_specific_packages(self):
        """Filters out the stuff we don't need by calling `search_packages`"""
        packages, _ = self.search_packages()

        for p_dict in packages:
            print(p_dict)
            pkg_id = p_dict["id"]
            subprocess.check_call(
                ["conan", "download", "-r", self.remote, f"{self.reference()}:{pkg_id}"],
                # stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
            )

    def upload_to_nrel(self):
        subprocess.check_call(
            ["conan", "upload", "-r", "nrel", "--all", "--parallel", "--no-overwrite", "all", self.reference()],
            # stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
        )

    def upload_specific_packages_to_nrel(self, arch_only=None, compiler_version_only=None):
        """Filters out the stuff we don't need by calling `search_packages(local_cache=True)`
        And upload only the packages that matches
        """
        if arch_only is None and compiler_version_only is None:
            raise ValueError("Provide at least one filter!")
        packages, _ = self.search_packages(
            arch_only=arch_only, compiler_version_only=compiler_version_only, local_cache=True
        )

        for p_dict in packages:
            print(p_dict)
            pkg_id = p_dict["id"]
            args = [
                "conan",
                "upload",
                "-r",
                "nrel",
                "--all",
                "--parallel",
                "--no-overwrite",
                "all",
                f"{self.reference()}:{pkg_id}",
            ]
            print(" ".join(args))
            subprocess.check_call(
                args,
                # stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
            )

    def package_dir(self):
        p = CONAN_CACHE / f"{self.name}/{self.version}"
        if self.user is not None:
            p /= f"{self.user}/{self.channel}"
        else:
            p /= "_/_"
        p /= "package"
        return p

    def cleanup_skipped_packages(self, remote=None, skip_shared=False, arch_only=None):
        """if remote is none, cleans up your local cache"""
        _, skipped_packages = self.search_packages(skip_shared=skip_shared, arch_only=arch_only)
        for p_dict in skipped_packages:
            pkg_id = p_dict["id"]
            cmd_args = ["conan", "remove", "-f", self.reference(), "-p", pkg_id]
            if remote is not None:
                cmd_args += ["-r", remote]

            subprocess.run(cmd_args)

    def reference(self):
        s = f"{self.name}/{self.version}@"
        if self.user is not None:
            s += f"{self.user}/{self.channel}"
        s += f"#{self.revision}"
        return s

    def __repr__(self):
        return self.reference()

    def __eq__(self, other):
        return self.reference() == other.reference()

# Download packages

## Parse my local cache

In [ ]:
CONAN_CACHE = Path("~/.conan/bump_deps/").expanduser()

pkg_infos = []
for p in CONAN_CACHE.glob("**/metadata.json"):
    pkg_infos.append(PkgInfo.from_metadata(p))
pkg_infos.sort(key=lambda p: p.name)

In [ ]:
len(pkg_infos)

In [ ]:
pkg_infos

## Compare with last time it was run

In [ ]:
old_refs = [
    "b2/4.8.0@#012527a73298c09865ac86e6921b8bc9",
    "benchmark/1.6.1@#94c40ebf065e3b20cab6a4f1b03a65fe",
    "bison/3.7.6@#de3449489624dbc45cfb8a868818def8",
    "boost/1.79.0@#f664bfe40e2245fa9baf1c742591d582",
    "bzip2/1.0.8@#b056f852bd2d5af96fc7171aadfd6c0b",
    "cpprestsdk/2.10.18@#df2f6ac88e47cadd9c9e8e0971e00d89",
    "flex/2.6.4@#e4696e6333f2d486e7073de6a9559712",
    "fmt/8.1.1@#b3e969f8561a85087bd0365c09bbf4fb",
    "gdbm/1.19@#c420bc00f3cc629aef665fdc3100f926",
    "geographiclib/1.52@#76536a9315a003ef3511919310b2fe37",
    "gmp/6.2.1@#2011237c81178d014a4c08ae40cfe0cb",
    "gnu-config/cci.20210814@#58573fa18a083c1ccb883b392b826bb2",
    "gtest/1.11.0@#8aca975046f1b60c660ee9d066764d69",
    "jsoncpp/1.9.5@#536d080aa154e5853332339bf576747c",
    "libbacktrace/cci.20210118@#b707cfa5d717e9e1401017c741ad8f6c",
    "libffi/3.4.2@#4121e32bfd908d32864cb97643b2b5a9",
    "libiconv/1.16@#eae489614aa6b1b8ca652cc33d3c26a9",
    "libyaml/0.2.5@#5bdf6971928f655a994646f8dbb221e4",
    "m4/1.4.19@#d9741f0aa0ac45e6b54a59f79e32ac81",
    "minizip/1.2.12@#0b5296887a2558500d0323c6c94c8d02",
    "nlohmann_json/3.9.1@#304649bcd7dae8fa24a2356e9437f9ad",
    "openssl/1.1.1o@#213dbdeb846a4b40b4dec36cf2e673d7",
    "openstudio_ruby/2.7.2@nrel/testing#98444b7bc8d391ea1521d7f79d4d4926",
    "pcre/8.45@#9158a180422a0d4dc01c668cbee37100",
    "pugixml/1.12.1@#5a39f82651eba3e7d6197903a3202e21",
    "readline/8.1.2@#ae31d1d71b027b0fe35903eb6c2e8333",
    "ruby_installer/2.7.3@bincrafters/stable#90fad7a169f6cb267c3e2e6aee106566",
    "sqlite3/3.38.5@#010911927ce1889b5cf824f45e7cd3d2",
    "stb/20200203@#cba8fa43f7a0ea5389896744664823c9",
    "swig/4.0.2@#9fcccb1e39eed9acd53a4363d8129be5",
    "termcap/1.3.1@#733491d29bb22e81d06edaf78c6727c9",
    "tinygltf/2.5.0@#c8b2aca9505e86312bb42aa0e1c639ec",
    "websocketpp/0.8.2@#3fd704c4c5388d9c08b11af86f79f616",
    "zlib/1.2.12@#3b9e037ae1c615d045a06c67d88491ae",
]
old_pkgs = [PkgInfo.from_str(x) for x in old_refs]

strawberryperl is a build_requires of openssl
nasm is a build_requires of openssl
winflexbison is a build_requires of swig
autoconf is a requires of automake
automake is a build_requires of bison (which is always a build_requires, and one of swig)
msys2 is always a build_requires (of automake, and of swig)

=> if we prebuild swig and openssl, these shouldn't be needed

### test bed a single package (boost)

## Compare packages I have in my cache with the conan.lock produced when building OS

In [ ]:
conan_lock = "/Users/julien/Software/Others/OS-build-bump/conan.lock"
with open(conan_lock, "r") as f:
    conan_lock_data = json.load(f)

In [ ]:
conan_lock_data.keys()

In [ ]:
pkg_infos_lock = []
for k, node in conan_lock_data["graph_lock"]["nodes"].items():
    if not "ref" in node:
        print(f"{k=} has no ref (it's node 0, that's normal)")
        continue
    pkg_infos_lock.append(PkgInfo.from_str(reference=node["ref"]))

In [ ]:
pkg_infos_lock

In [ ]:
set(x.reference() for x in pkg_infos) - set(x.reference() for x in pkg_infos_lock)

In [ ]:
set(x.reference() for x in pkg_infos_lock) - set(x.reference() for x in pkg_infos)

## Download all packages

In [ ]:
for pkg_info in pkg_infos:
    if pkg_info.name == "openstudio_ruby":
        continue

    print(pkg_info.name)

    # Filter before downloading:
    # keep_packages, skip_packages = pkg_info.search_packages()
    # pkg_info.download_specific_packages()

    # download_all has the benefit of running in parallel... so it's faster provided you have a good connection
    # We'll clean it up later
    pkg_info.download_all()
    print("\n")

# Clean up unwanted binaries

In [ ]:
for pkg_info in pkg_infos:
    if pkg_info.name == "openstudio_ruby":
        continue

    print(pkg_info.name)
    pkg_info.cleanup_skipped_packages(skip_shared=True)
    print("\n")

# Upload to NREL

In [ ]:
for pkg_info in pkg_infos:
    if pkg_info.name == "openstudio_ruby":
        continue
    print(pkg_info.name)
    pkg_info.upload_to_nrel()
    print("\n")

# In one go

# Windows

## Upload unusual configurations: the win32 and MSVC 17 stuff

That's not part of conancenter

In [ ]:
# Not sure whether I should upload those or not...?
extra_win_build_requires = ["autoconf", "automake", "msys2", "nasm", "strawberryperl", "winflexbison"]

In [ ]:
pkg_infos_regular = [x for x in pkg_infos if not x.name in extra_win_build_requires]
len(pkg_infos_regular)

In [ ]:
pkg_infos_regular

In [ ]:
for pkg_info in pkg_infos_regular:
    keep_packages, _ = pkg_info.search_packages(
        verbose=False,
        local_cache=True,
        compiler_version_only="17",
        # arch_only='x86'
    )
    if not keep_packages:
        print(pkg_info)
    else:
        print(f"Found {len(keep_packages)} for {pkg_info}")

In [ ]:
for pkg_info in pkg_infos_regular:
    # if pkg_info.name == 'boost':
    #    continue
    pkg_info.upload_specific_packages_to_nrel(
        # arch_only='x86',
        compiler_version_only="17"
    )
    done.append(pkg_info)

In [ ]:
print("DONE")

## Upload the windows specific build_requires

In [ ]:
extra_win_build_requires

In [ ]:
pkg_infos_win_build_requires = [x for x in pkg_infos if x.name in extra_win_build_requires]
pkg_infos_win_build_requires

In [ ]:
for pkg_info in pkg_infos_win_build_requires:
    print(pkg_info.name)
    pkg_info.download_all()
    pkg_info.cleanup_skipped_packages(skip_shared=True)
    pkg_info.upload_to_nrel()
    print("\n")